In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.decomposition import KernelPCA


In [2]:
y_raw=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train_label.csv", sep=',')
def variable_extracter(hist):
    loss = hist.history['loss'][0]
    acc=hist.history['accuracy'][0]

    return loss, acc

def scalepca_pipeline(x, scaler):
    x=scaler.transform(x)
    #x=pca.transform(x)
    return x

def batch_generater(filenum, scaler, nrows=None, rate=0.9, root=r"C:\Users\ksi03\Downloads\RP_data\train\\",):
    y_f=[]
    if nrows==None:
        x_raw=pd.read_csv(root+str(filenum)+".csv", sep=',').values
        #x_f=scalepca_pipeline(x_raw, scaler, pca)
        x_f=scaler.transform(x_raw)
        
        y=y_raw.iloc[filenum, 1]
        loc=10
        for _ in range(loc):
            y_f.append(198)

        for i in range(x_f.shape[0]-loc):
            y_f.append(y)
    else:
        x_raw=pd.read_csv(root+str(filenum)+".csv", sep=',', nrows=nrows).values
        #x_f=scalepca_pipeline(x_raw, scaler, pca)
        x_f=scaler.transform(x_raw)
        
        y=y_raw.iloc[filenum, 1]
        loc=10
        for stat_a in range(loc):
            y_f.append(198)

        for stat_b in range(nrows-loc):
            y_f.append(y)

        
    y_f=np.asarray(y_f)
    return x_f, y_f


def scaler_pca_generater( nrows, ref_filenum=100, dim=1000):
    #trainset인 0~745 이내에서 샘플을 100개 추출해 scaler와 pca 제작
    filelist=random.sample(range(0, 745), ref_filenum)
    scaler=MinMaxScaler()
    kpca = KernelPCA(n_components=dim, kernel='rbf')
    x_sample=[]
    i=False
    for num in filelist:
        if num!=30:
            if i!=False:
                tmp=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train\\"+str(num)+".csv", sep=',', nrows=nrows)
                tmp=tmp.iloc[10:]
                x_sample=data_raw=pd.concat([x_sample, tmp], ignore_index=True)
            else:
                x_sample=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train\\"+str(num)+".csv", sep=',', nrows=nrows)
        
    scaler.fit(x_sample)
    #kpca.fit(x_sample)
    kpca=False
    
    del x_sample
    return filelist, scaler


In [3]:
_, scaler=scaler_pca_generater(nrows=30, ref_filenum=300)

In [4]:
#828개의 dataset, 745개는 train, 43개는 valid, 40개는 test
#valid, train=상주변수 
#train=가변변수
y_raw=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train_label.csv", sep=',')
for i in range(746, 788):
    if i==746: 
        x_t, y_t=batch_generater(i, scaler,  nrows=30)
        x_t=x_t[10:]
        y_t=y_t[10:]
    else: 
        x_t2, y_t2=batch_generater(i, scaler, nrows=30)
        x_t2=x_t2[10:]
        y_t2=y_t2[10:]
        
        x_t=np.concatenate((x_t, x_t2))
        y_t=np.concatenate((y_t, y_t2))
        
y_valid=y_t
x_valid=x_t



In [5]:
for i in range(789, 828):
    if i==746: 
        x_t, y_t=batch_generater(i, scaler, nrows=30)
        x_t=x_t[10:]
        y_t=y_t[10:]
    else: 
        x_t2, y_t2=batch_generater(i, scaler, nrows=30)
        x_t2=x_t2[10:]
        y_t2=y_t2[10:]
        x_t=np.concatenate((x_t, x_t2))
        y_t=np.concatenate((y_t, y_t2))
        
y_test=y_t
x_test=x_t

In [6]:
x_valid=x_valid.reshape(x_valid.shape[0], 1, x_valid.shape[1])


In [7]:
x_test=x_test.reshape(x_test.shape[0], 1, x_test.shape[1])

print(x_valid.shape)
print(y_valid.shape)
print(x_test.shape)
print(y_test.shape)

(840, 1, 5122)
(840,)
(1620, 1, 5122)
(1620,)


In [8]:
input_steps=1
hidden_dim=10
input_dim=5122
init=tf.keras.initializers.he_uniform()
reg=tf.keras.regularizers.l2(l=0.005)
droprate = 0.3
act='relu'

L_model = tf.keras.Sequential(
[
tf.keras.layers.Input(shape=(input_steps, input_dim)),
tf.keras.layers.GRU(hidden_dim ,return_state=False, kernel_initializer=init, kernel_regularizer=reg),
tf.keras.layers.Dropout(droprate),
#tf.keras.layers.Dense(300, activation=act),
tf.keras.layers.Dense(30),
tf.keras.layers.LeakyReLU(alpha=0.05),
tf.keras.layers.Dropout(droprate),
tf.keras.layers.Dense(10),
tf.keras.layers.LeakyReLU(alpha=0.05),
tf.keras.layers.Dropout(droprate),
tf.keras.layers.GaussianNoise(0.05),
tf.keras.layers.Dense(198, activation='softmax')])


L_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
            #optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss= 'sparse_categorical_crossentropy',
              metrics= ['accuracy'])

In [9]:

_=True
for file in range(746):
    if file!=30:
        x_t, y_t=batch_generater(file, scaler, nrows=100)
        x_t=x_t[10:]
        y_t=y_t[10:]

        if _==True:
            x_train, y_train=x_t, y_t
        else:
            x_train=np.concatenate((x_train, x_t))
            y_train=np.concatenate((y_train, y_t))


        print('file{} is complete'.format(file))
        _=False
    
    
x_train=x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
print(x_train.shape)
print(y_train.shape)

file0 is complete
file1 is complete
file2 is complete
file3 is complete
file4 is complete
file5 is complete
file6 is complete
file7 is complete
file8 is complete
file9 is complete
file10 is complete
file11 is complete
file12 is complete
file13 is complete
file14 is complete
file15 is complete
file16 is complete
file17 is complete
file18 is complete
file19 is complete
file20 is complete
file21 is complete
file22 is complete
file23 is complete
file24 is complete
file25 is complete
file26 is complete
file27 is complete
file28 is complete
file29 is complete
file31 is complete
file32 is complete
file33 is complete
file34 is complete
file35 is complete
file36 is complete
file37 is complete
file38 is complete
file39 is complete
file40 is complete
file41 is complete
file42 is complete
file43 is complete
file44 is complete
file45 is complete
file46 is complete
file47 is complete
file48 is complete
file49 is complete
file50 is complete
file51 is complete
file52 is complete
file53 is complete
fil

file417 is complete
file418 is complete
file419 is complete
file420 is complete
file421 is complete
file422 is complete
file423 is complete
file424 is complete
file425 is complete
file426 is complete
file427 is complete
file428 is complete
file429 is complete
file430 is complete
file431 is complete
file432 is complete
file433 is complete
file434 is complete
file435 is complete
file436 is complete
file437 is complete
file438 is complete
file439 is complete
file440 is complete
file441 is complete
file442 is complete
file443 is complete
file444 is complete
file445 is complete
file446 is complete
file447 is complete
file448 is complete
file449 is complete
file450 is complete
file451 is complete
file452 is complete
file453 is complete
file454 is complete
file455 is complete
file456 is complete
file457 is complete
file458 is complete
file459 is complete
file460 is complete
file461 is complete
file462 is complete
file463 is complete
file464 is complete
file465 is complete
file466 is complete


In [15]:
import gc
##러닝
current_cost_train = 1000 #초기 값으로 높은 값 부여
current_cost_valid = 1000 #초기 값으로 높은 값 부여
known_best = 1000 #초기 값으로 높은 값 부여
full_step = 11
best_model=0
acc=0
trainValues=[]
validValues=[]


for file in range(100): #100회의 파일
    if file!=None:
        train_hist = L_model.fit(x_train, y_train, batch_size=300, verbose=1)
        loss, acc = variable_extracter(train_hist)

        gc.collect()
        
        if (file > 0 and file % 10 == 0): #매 10번째 학습 성능 값 출력
            print("[step: {}]'s cost = {}   acc={}".format(file, loss, acc)) 
            if (file >= 0): #1000번째부터 검증데이터로 모델 성능 평가
                current_cost_valid = L_model.evaluate(x_valid, y_valid,batch_size=100, verbose=0)
                gc.collect()
                trainValues.append(loss) #학습 곡선을 출력하기 위해 현재 값 저장
                validValues.append(current_cost_valid[0]) #검증 곡선을 출력하기위해 현재 값 저장

                if (current_cost_valid[0] < known_best): #검증 성능이 이전보다 좋을 경우 현재 모델 저장
                    known_best = current_cost_valid[0]
                    best_model=L_model
                    print("New best known cost = {}, acc={}".format(known_best, current_cost_valid[1]))

        #print("학습루틴 {}회 종료".format(file))


##
print("--- 학습 종료 ---")

label_model=best_model

Train on 67050 samples
67050/67050 [==============================] - 7s 97us/sample - loss: 3.3018 - accuracy: 0.1926
Train on 67050 samples
67050/67050 [==============================] - 7s 99us/sample - loss: 3.3760 - accuracy: 0.1919
Train on 67050 samples
67050/67050 [==============================] - 6s 86us/sample - loss: 3.3073 - accuracy: 0.1962
Train on 67050 samples
67050/67050 [==============================] - 6s 90us/sample - loss: 3.2742 - accuracy: 0.2005
Train on 67050 samples
67050/67050 [==============================] - 6s 88us/sample - loss: 3.3021 - accuracy: 0.2003
Train on 67050 samples
67050/67050 [==============================] - 6s 85us/sample - loss: 3.2913 - accuracy: 0.2017
Train on 67050 samples
67050/67050 [==============================] - 6s 88us/sample - loss: 3.2594 - accuracy: 0.1997
Train on 67050 samples
67050/67050 [==============================] - 6s 88us/sample - loss: 3.2414 - accuracy: 0.2022
Train on 67050 samples
67050/67050 [============

67050/67050 [==============================] - 5s 81us/sample - loss: 3.1285 - accuracy: 0.2194
Train on 67050 samples
67050/67050 [==============================] - 6s 86us/sample - loss: 3.1491 - accuracy: 0.2181
Train on 67050 samples
67050/67050 [==============================] - 5s 81us/sample - loss: 3.1363 - accuracy: 0.2159
Train on 67050 samples
67050/67050 [==============================] - 6s 84us/sample - loss: 3.1127 - accuracy: 0.2177
Train on 67050 samples
67050/67050 [==============================] - 6s 85us/sample - loss: 3.1706 - accuracy: 0.2109
Train on 67050 samples
67050/67050 [==============================] - 5s 80us/sample - loss: 3.0909 - accuracy: 0.2155
Train on 67050 samples
67050/67050 [==============================] - 6s 83us/sample - loss: 3.0497 - accuracy: 0.2206
[step: 70]'s cost = 3.0496731236477026   acc=0.22062639892101288
Train on 67050 samples
67050/67050 [==============================] - 6s 83us/sample - loss: 3.1044 - accuracy: 0.2208
Train 

In [16]:
import gc
gc.collect()

0

In [17]:
validValues

[4.695433662051246,
 5.100802455629621,
 4.721762975056966,
 4.987508648917789,
 4.886150632585798,
 5.350240815253485,
 5.538573662439982,
 5.910315161659604,
 6.34857173760732]

In [19]:
#x_train.shape[0]
#이렇게 돌린 모델과 다른 모델 비교하기
ref='100b_1'
L_model.save(r"C:\\Users\ksi03\model\mod2.h5")

In [20]:
L_model=tf.keras.models.load_model(r"C:\\Users\ksi03\model\mod2.h5")
testcost=L_model.evaluate(x_test, y_test, verbose=0)
print(testcost[0])
print(testcost[1])
y_tmp=[]
y_final=[]
_=True
for num in range(828, 1548):
    if num!=1154 and num!=1168:
        
        x_raw=pd.read_csv(r'C:\Users\ksi03\Downloads\RP_data\test\\'+str(num)+".csv", sep=',', nrows=30).values
        x_raw=x_raw[10:]
        x_real=scalepca_pipeline(x_raw, scaler)
        x_real=x_real.reshape(x_real.shape[0], 1, x_real.shape[1])
        
        y_real=L_model.predict(x_real,batch_size=1, verbose=0) 
        
        y_tmp=[]
        for i in range(y_real.shape[0]):
            y_tmp.append(np.argmax((y_real[i])))
             

        y_final.append([num, str(np.argmax(np.bincount(np.array(y_tmp))))])
        print(num)  
    else:
        y_final.append([num, str(np.argmax(np.bincount(np.array(y_tmp))))])
        print(num)  
                         
            
      


7.396516463491651
0.245679
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056


In [21]:
y_tmp=np.array(y_final)
print(y_tmp.shape)


(720, 2)


In [34]:
y_tmp

array([['828', '111'],
       ['829', '118'],
       ['830', '110'],
       ...,
       ['1545', '168'],
       ['1546', '110'],
       ['1547', '110']], dtype='<U11')

In [35]:
y=pd.DataFrame(y_tmp, columns=['num','label'])


In [37]:
y_dummy = pd.get_dummies(y, columns=['label'])
y_dummy.shape

(720, 41)

In [38]:
y_dummy.to_csv('.\y2.csv', index=False)